'1': 필요한 라이브러리 import 하고 dataset이 있는 path 잡아주고 CT폴더 경로 및 RTst로부터 얻은 Contour파일 load함.

In [46]:
import os
import numpy as np
import pydicom
from numpngw import write_png

#from PIL import Image #PIL은 8bit image만 다룰수 있음.
#https://stackoverflow.com/questions/62739851/convert-rgb-arrays-to-pil-image
#CT폴더 불러오기
#Contour파일 불러오기
basepath = 'D:\CodeMaster\w7_dataset'
CT_folder_path = basepath + '\P071_HN71_CT_2019-02-01_114956_._Neck.3.0_n172__00001'
ctr_voxel = np.load('D:\CodeMaster\w7voxels\p071rtst\p071voxel_ctr.npy')

'2': CT정보를 불러와서 Instance Number에 맞추어 정렬(머리부터 시작)

In [47]:
def GetCTinfo(CT_folder_path):
    CT_dcm_files = []
    fileidx = []
    for top, dir, file in os.walk(CT_folder_path):
        for filename in file:
            if filename[-3:] != 'dcm':
                continue
            temp_ct_slice = pydicom.dcmread(CT_folder_path + '/' + filename)
            CT_dcm_files.append(temp_ct_slice)
            idxnum = temp_ct_slice.InstanceNumber
            fileidx.append(idxnum)
        break

    fileidx = np.array(fileidx)
    fileorder = np.argsort(fileidx)

    sorted_CT_file = []
    
    for i in range(len(fileidx)):
        sorted_CT_file.append(CT_dcm_files[fileorder[i]])
        
    return sorted_CT_file, len(fileidx)
    
sorted_CT_file, zsize = GetCTinfo(CT_folder_path)

'3': 각 slice 마다 Windowing 해줄 함수.

In [48]:
def GetWindowMaxandMin(ct_dicomdata):

    if type(ct_dicomdata.WindowCenter) == pydicom.valuerep.DSfloat:
        window_center = ct_dicomdata.WindowCenter
        window_width = ct_dicomdata.WindowWidth
    else:
        window_center = ct_dicomdata.WindowCenter[0]
        window_width = ct_dicomdata.WindowWidth[0]

    img_max = window_center + window_width // 2
    img_min = window_center - window_width // 2
    
    return img_max, img_min
    

'4': 변환용 Slope와 Intercept구해줄 함수.

In [49]:
def GetSlopeandIntercept(ct_dicomdata):
    slope = float(ct_dicomdata.RescaleSlope)
    intercept = float(ct_dicomdata.RescaleIntercept)
    return slope, intercept

'5': Gray에서 RGB로 변환하기 위해 2차원 배열을 3개 쌓음(ex: 255 ->[255, 255, 255])

In [50]:
def GraytoRGB(arr_slice):
    pixel_rgb = np.stack((arr_slice, )*3, axis = -1)
    return pixel_rgb

'6': 위의 두 함수를 이용하여 Window Center, Window Width와 기존 픽셀값을 HU unit으로 Rescale해주는 값을 얻어 slice array마다 Window를 적용해줌. 그후 16bit에 맞추어 값을 array값을 변환후 RGB화 시켜줌, 변환시 overflow 방지를 위해 int32로 선언
(최댓값이 65536까지 가야하므로.)

In [51]:
def MakeCTvoxel(sorted_CT_file, zsize):
    CT_voxel = np.zeros((zsize, 512, 512, 3), 'int32')

    for i in range(len(sorted_CT_file)):
        ct_dicomdata = sorted_CT_file[i]
        convert_coeff = 2**(ct_dicomdata.BitsAllocated-1)
        slope, intercept = GetSlopeandIntercept(ct_dicomdata)
        img_max, img_min = GetWindowMaxandMin(ct_dicomdata)
        temp_arr_slice = slope * ct_dicomdata.pixel_array + intercept
        temp_arr_slice = temp_arr_slice.astype('int32')

        temp_arr_slice[temp_arr_slice < img_min] = img_min
        temp_arr_slice[temp_arr_slice > img_max] = img_max
        
        temp_arr_slice *= round(convert_coeff/(img_max-img_min))
        CT_voxel[i] = GraytoRGB(temp_arr_slice)

    return CT_voxel

CT_voxel = MakeCTvoxel(sorted_CT_file, zsize)


'7': HU unit은 음의 부호도 존재하나 나의 코드에서 png로 저장하기 위해서는 uint16으로 저장해야함. 따라서 위에서 int32으로 수정했던 부분을 uint16으로 저장. 또한 max값을 모두 픽셀의 최대치로, min값을 최소치로 바꿔줌.

In [52]:
def RefineCTvoxel(sorted_CT_file, CT_voxel):
    print(np.min(CT_voxel))
    print(np.max(CT_voxel))
    convert_coeff = 2**(sorted_CT_file[0].BitsAllocated-1)
    CT_voxel += int(convert_coeff) - abs(np.min(CT_voxel))
    CT_voxel[CT_voxel <= np.min(CT_voxel)] = 0
    CT_voxel[CT_voxel >= np.max(CT_voxel)] = 2*convert_coeff - 1
    CT_voxel = CT_voxel.astype('uint16')

    return CT_voxel

CT_voxel = RefineCTvoxel(sorted_CT_file, CT_voxel)

-12690
20210


'8': CT는 머리쪽부터 저장된게 default임. 그러나 RTst로부터 얻은 Contour는 배쪽부터 저장되어 있기에 둘을 overlay해주려면 RTst혹은 CT를 reverse시켜야함. 

In [53]:
def AddContourtoCT(CT_voxel, ctr_voxel):
    ctr_voxel = np.flip(ctr_voxel, axis = 0)

    #ctr_mask = np.zeros_like(ctr_voxel, dtype=bool)
    #ctr_mask = np.where(ctr_voxel != [0, 0, 0], [True, True, True], ctr_mask)
    #CT_ctr_voxel = np.where(ctr_mask == [True, True, True], ctr_voxel, CT_voxel)
    CT_ctr_voxel = CT_voxel + ctr_voxel//2
    
    return CT_ctr_voxel

CT_ctr_voxel = AddContourtoCT(CT_voxel, ctr_voxel)

'9': 사진 및 voxel 저장부분.

In [54]:
np.save('D:\CodeMaster\w7voxels\p071ct\p071CTvoxel', CT_ctr_voxel)
for k in range(len(CT_ctr_voxel)):
    write_png('D:\CodeMaster\w7voxels\p071ct\p071ct%d.png' %(k+1), CT_ctr_voxel[k])